## purpose: 一パス内でのテストデータの数とsubmissionに必要なデータ数を比較する。もしtestデータのほうが多いならば、余分なテストデータで予測を向上させる特徴量が作成できる

In [1]:
import sys
sys.path.append("..")

import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil

from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength
from io_f import read_data_file
from visualize_f import visualize_trajectory, visualize_heatmap
from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength
from main import extract_wifi_rssi, extract_wifi_count
import processing
import os

sns.set_theme()

%load_ext autoreload
%autoreload 2

Duplicate key in file PosixPath('/Users/takahashigenki/.matplotlib/stylelib/genki.mplstyle'), line 176 ('font.family         : IPAexGothic')


In [2]:
feature_dir = "../resource/wifi_features"
# the metric used in this competition
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

# get our train and test files
train_files = sorted(glob.glob(os.path.join(feature_dir, 'train/*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, 'test/*_test.csv')))
ssubm = pd.read_csv('../sample_submission.csv', index_col=0)

In [14]:
# pred_df = pd.DataFrame([[v.split("_")[0],v.split("_")[1], v.split("_")[2]] for v in ssubm.index])
# display(pred_df.head())

# pred_df.groupby([0,1]).count()

,0,1,2
0,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000000009
1,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000009017
2,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000015326
3,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000018763
4,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000022328


2
0                        1                           
5a0546857ecc773753327266 046cfa46be49fc10834815c6   9
                         05d052dde78384b0c543d89c   8
                         0c06cc9f21d172618d74c6c8  14
                         146035943a1482883ed98570  10
                         1ef2771dfea25d508142ba06   8
...                                                ..
5dc8cea7659e181adb076a3f e83a1c294b5d138339149fcb  40
                         ee40c6e25fe024d5da451909  31
                         f9f99869dad1b2021ec7a7e2  25
                         fd4eee038c0ad7f0179e337e  15
                         fd64de8c4a2fc5ebb0e9f412  17

[626 rows x 1 columns]

'../test/6b6d91a5b97ed3c40c1411a7.txt'

`'../test/6b6d91a5b97ed3c40c1411a7.txt'` このパスに置いては、testデータ数は3万行あるが、submissionの方は15件のみである。なので、データ数はかなり多いと考えて良い。